# **Import thư viện cần thiết**

In [462]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import itables
from itables import init_notebook_mode
from itables import show

# **Nạp dữ liệu**

In [463]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

# **Xử lý dữ liệu và tạo đặc trưng mới**

## xử lý đặc trưng Name bằng cách tạo một đặc trưng mới tên 'Title' lưu lại danh từ xưng hô của các tên

In [464]:
train['Title'] = train['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
test['Title'] = test['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
train.drop(['Name'],inplace=True,axis=1)
test.drop(['Name'],inplace=True,axis=1)

In [465]:
train['Title'].value_counts()

Title
Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Col           2
Mlle          2
Major         2
Ms            1
Mme           1
Don           1
Lady          1
Sir           1
Capt          1
Countess      1
Jonkheer      1
Name: count, dtype: int64

In [466]:
test['Title'].value_counts()

Title
Mr        240
Miss       78
Mrs        72
Master     21
Col         2
Rev         2
Ms          1
Dr          1
Dona        1
Name: count, dtype: int64

Giảm bớt các danh hiệu bằng từ điển

In [467]:
Title_Dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Dona": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

train['Title'] = train['Title'].map(Title_Dictionary)
test['Title'] = test['Title'].map(Title_Dictionary)


In [468]:
print('Title null trong train: ',train['Title'].isna().sum())
print('Title null trong test: ',test['Title'].isna().sum())

Title null trong train:  0
Title null trong test:  0


In [469]:
show(train)

Loading ITables v2.5.2 from the internet... (need help?)


In [470]:
show(test)

Loading ITables v2.5.2 from the internet... (need help?)


## Xử lý đặc trưng Age

Tìm trung vị của Age sau khi nhóm thêm Pclass và Sex

In [471]:
print('Số đặc trưng Age null trong train: ',train['Age'].isna().sum())
print('Số đặc trưng Age null trong test: ',test['Age'].isna().sum())

Số đặc trưng Age null trong train:  177
Số đặc trưng Age null trong test:  86


In [472]:
train['Age'] = train.groupby(['Pclass','Sex','Title'])['Age'].transform(lambda x : x.fillna(x.median()))

In [473]:
print('Số đặc trưng Age null trong train: ',train['Age'].isna().sum())

Số đặc trưng Age null trong train:  0


Để tránh data leekage tập test thì ta sẽ dùng các giá trị thiếu của tập train để gắn vào test

In [474]:
grouped_train = train.groupby(['Pclass','Sex','Title'])
train_age = grouped_train['Age'].median()
train_age

Pclass  Sex     Title  
1       female  Miss       30.0
                Mrs        40.0
                Officer    49.0
                Royalty    40.5
        male    Master      4.0
                Mr         40.0
                Officer    51.0
                Royalty    40.0
2       female  Miss       24.0
                Mrs        31.5
        male    Master      1.0
                Mr         31.0
                Officer    46.5
3       female  Miss       18.0
                Mrs        31.0
        male    Master      4.0
                Mr         26.0
Name: Age, dtype: float64

In [475]:
mapped_test = test.set_index(['Pclass', 'Sex', 'Title']).index.map(train_age)
mapped_ages_for_test = pd.Series(mapped_test, index=test.index)
test['Age'] = test['Age'].fillna(mapped_ages_for_test)

In [476]:
print('Số đặc trưng Age null trong test: ',test['Age'].isna().sum())

Số đặc trưng Age null trong test:  0


### Tạo 2 đặc trưng để xem người đó phải trẻ em hay là người mẹ hay không

In [477]:
dfs = (train,test)
for df in dfs:
    df['IsChild'] = (df['Age'] < 12).astype(int)
    df['IsMother'] = ((df['Sex'] == 'female') & (df['Parch'] > 0) & (df['Age'] > 18) & (df['Title'] == 'Mrs')).astype(int)

## Xử lý đặc trưng SibSp và Parch

### Tạo một đặc trưng mới tên FamilySize. FamilySize = SibSp + Parch + 1

In [478]:
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

In [479]:
show(train)

Loading ITables v2.5.2 from the internet... (need help?)


In [480]:
train.shape

(891, 15)

## Xử lý đặc trưng Embarked và Fare

In [481]:
dfs = [train,test]
for df in dfs:
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0]) #lấy giá trị xuất hiện nhiều nhất(Mode) để gán vào giá trị thiếu trong đặc trưng Embarked
    df['Fare'] = df.groupby('Pclass')['Fare'].transform(lambda x: x.fillna(x.median()))

In [482]:
print('Giá trị Embarked xuất hiện nhiều nhất: ',df['Embarked'].mode()[0])

Giá trị Embarked xuất hiện nhiều nhất:  S


## Xử lý đặc trưng Ticket

In [483]:
def process_ticket(df):
    def clean_ticket_prefix(ticket):
        cleaned_ticket = (
            ticket.replace('.', '')
            .replace('/', '')
            .strip()
            .split()
        )
        
        prefix_list = [
            item.strip() 
            for item in cleaned_ticket 
            if not item.strip().isdigit()
        ]

        if prefix_list:
            return prefix_list[0]
        else: 
            return 'XXX'
    
    # Tạo cột mới và trả về DataFrame
    df['Ticket_Prefix'] = df['Ticket'].apply(clean_ticket_prefix)
    df.drop(['Ticket'],axis=1,inplace=True)
    return df


In [484]:
train = process_ticket(train)
test = process_ticket(test)

In [485]:
train['Ticket_Prefix'].value_counts()

Ticket_Prefix
XXX        661
PC          60
CA          41
A5          21
SOTONOQ     15
STONO       12
WC          10
A4           7
SCPARIS      7
STONO2       6
SOC          6
C            5
FCC          5
LINE         4
SCParis      4
SOPP         3
SCAH         3
WEP          3
PP           3
PPP          2
SOTONO2      2
SWPP         2
SP           1
SCA4         1
SCOW         1
SOP          1
Fa           1
AS           1
SC           1
FC           1
CASOTON      1
Name: count, dtype: int64

In [486]:
show(train)

Loading ITables v2.5.2 from the internet... (need help?)


In [487]:
show(test)

Loading ITables v2.5.2 from the internet... (need help?)


## Xử lý đặc trưng Cabin

In [488]:
train_cabin, test_cabin = set(), set()
for char in train['Cabin']:
    try:
        train_cabin.add(char[0])
    except:
        train_cabin.add('U')

for char in test['Cabin']:
    try:
        test_cabin.add(char[0])
    except:
        test_cabin.add('U')

In [489]:
train_cabin

{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'U'}

In [490]:
test_cabin

{'A', 'B', 'C', 'D', 'E', 'F', 'G', 'U'}

In [491]:
dfs = [train,test]
for df in dfs:
    df['Cabin'] = df['Cabin'].fillna('U')
    df['Cabin'] = df['Cabin'].map(lambda x: x[0])

In [492]:
show(train)

Loading ITables v2.5.2 from the internet... (need help?)


In [493]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             0
Cabin            0
Embarked         0
Title            0
IsChild          0
IsMother         0
FamilySize       0
Ticket_Prefix    0
dtype: int64

In [494]:
test.isna().sum()

PassengerId      0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             0
Cabin            0
Embarked         0
Title            0
IsChild          0
IsMother         0
FamilySize       0
Ticket_Prefix    0
dtype: int64

## Xử lý lại dữ liệu

### Xóa các đặc trưng không dùng

In [495]:
dfs = [train,test]
for df in dfs:
    df.drop(['PassengerId'],inplace=True,axis=1)
    df.drop(['SibSp'],inplace=True,axis=1)
    df.drop(['Parch'],inplace=True,axis=1)

### Tạo 2 đặc trưng 
**Age*Pclass**: Kết hợp tuổi và hạng vé – phản ánh “tầng lớp xã hội theo tuổi”.

**Fare_Pclass**: Giá vé chia theo hạng vé.

In [496]:
dfs = [train,test]
for df in dfs:
    df['Age*Pclass'] = df['Age'] * df['Pclass']
    df['Fare_Pclass'] = df['Fare'] / df['Pclass']

### Chuẩn hóa dữ liệu Age và Fare

In [497]:
dfs = [train,test]
for df in dfs:
    df['Fare'] = np.log1p(df['Fare'])
    df['Age'] = np.log1p(df['Age'])

In [498]:
show(train)

Loading ITables v2.5.2 from the internet... (need help?)


In [499]:
show(test)

Loading ITables v2.5.2 from the internet... (need help?)


In [500]:
nums =['Age','Fare','FamilySize','Age*Pclass','Fare_Pclass']
cats =['Pclass','Sex','Cabin','Embarked','Title','Ticket_Prefix','IsChild','IsMother']

train.drop(['Survived'],axis=1,inplace=True)
combined = pd.concat([train,test],ignore_index=True)

scaler = StandardScaler()
combined[nums] = scaler.fit_transform(combined[nums])

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False) 

# Chỉ chọn các cột phân loại từ combined
data_to_encode = combined[cats]

# Fit và Transform
encoded_data_array = encoder.fit_transform(data_to_encode)

# 2. Tạo DataFrame mới từ kết quả mã hóa
# Lấy tên các cột mới (ví dụ: 'Pclass_1', 'Sex_male', ...)
feature_names_out = encoder.get_feature_names_out(input_features=cats)

encoded_df = pd.DataFrame(
    encoded_data_array,
    columns=feature_names_out,
    index=combined.index # Đảm bảo index khớp với combined
)

# 3. Loại bỏ các cột phân loại gốc và ghép nối
# Loại bỏ các cột gốc đã được mã hóa
combined_dropped = combined.drop(columns=cats)

# Ghép nối DataFrame mới được mã hóa
combined = pd.concat([combined_dropped, encoded_df], axis=1)


In [510]:
combined.shape

(1309, 69)

In [502]:
show(combined)

Loading ITables v2.5.2 from the internet... (need help?)


# Lưu lại vào exps

## Tạo thư mục

In [503]:
exp_dir = "../exps"
if os.path.exists(exp_dir) == False:
    os.makedir(exp_dir,exist_ok=True)

save_dir = f"{exp_dir}/feature_Labels"
os.makedirs(save_dir,exist_ok=True)

## Lưu dữ liệu Features

In [504]:
combined.iloc[:891].to_csv(f'{save_dir}/train_final.csv',index=False)
combined.iloc[891:].to_csv(f'{save_dir}/test_final.csv',index=False)